# Product Onsite Analysis

In [2]:
import pandas as pd
from prompts import *
from policy_qa import *

/Users/jmbeck/PycharmProjects/PythonProject/MercuryInterview/src/language_models.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/Users/jmbeck/PycharmProjects/PythonProject/MercuryInterviewMercuryInterview.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Data

In [3]:
onsite_expenses = pd.read_excel('../data/Mercury Transactions Example Safe to Share - 20250211 - Send.xlsx')

In [4]:
onsite_expenses.shape

(165, 34)